In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
# from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.decomposition import PCA
import SimpleITK as sitk
import pandas as pd

import sys
np.set_printoptions(threshold=sys.maxsize)

## functions

In [4]:
## function definition

# function to convert DESI text file to numpy array
def DESI_txt2numpy(desi_text):
    """""
    input:
    desi_text - the location /name of the DESI text file
    output:
    peaks - 2D numpy array of DESI spectra with size [N_spectra x N_ions]
    mz - 1D numpy array of ions' m/z values [N_ions]
    dim_y - number of spectra in y axis
    dim_x - number of spectra in x axis
    (dim_y * dim_x = N_spectra)
    """""
    ##############################################
    # inner function for getting the number of spectra without buffering vthe data
    def num_lines_in_text(desi_text):

        # inner function for creating the generator
        def _count_generator(reader):
            b = reader(1024 * 1024)
            while b:
                yield b
                b = reader(1024 * 1024)
  
        with open(desi_text, 'rb') as read_obj:
            c_generator = _count_generator(read_obj.raw.read)
            # count each \n
            count = sum(buffer.count(b'\n') for buffer in c_generator)
            """Enters while loop, which continues as long as b (the chunk of data) is not empty. 
            This means that it will keep reading and yielding chunks of data until there's nothing left to read."""

        return count
    ##############################################
    
    print('file:', desi_text)

    total = num_lines_in_text(desi_text)
    print('spectra:', total-5)
    
    data = []
    with open(desi_text, 'r') as read_obj:
        for i,line in tqdm(enumerate(read_obj) , total=total):
            x = line.split()
            y = [float(num) for num in x]
            data.append(y)
            
    ind = np.argsort(data[3]) # data[3] has unsorted m/z values
    mz = np.take_along_axis(np.asarray(data[3]), ind, axis=0) # sort with indices

    x, y = [], []
    peaks = []
    for i in tqdm(range(4,len(data)-1)):
        x.append(data[i][1])
        y.append(data[i][2])
        p = np.asarray(data[i][3:-2])
        p = np.take_along_axis(p, ind, axis=0)
        p = np.expand_dims(p,axis=0)
        peaks.append(p)
    peaks = np.concatenate(peaks,axis=0)

    ## find desi data dimension
    t = np.asarray(x)
    t = np.abs(np.diff(t))
    dim_x = int(np.round(np.max(t)/np.min(t)))+1
    t = np.asarray(y)
    dim_y = int(np.round(np.abs(t[0]-t[-1])/np.max(np.abs(np.diff(t)))))+1
    print('image dimension:',dim_y, 'x', dim_x)
    
    return peaks, mz, dim_y, dim_x

# total ion current normalization (spectrum based)
def tic_normalize(peaks):
    tot_ion_cur = np.sum(peaks, axis=1)
    peaks_ticn = np.empty(peaks.shape)
    for i in range(len(peaks)):
        if tot_ion_cur[i]!=0:
            peaks_ticn[i] = peaks[i]/tot_ion_cur[i]
    return peaks_ticn

# normalize each ion in whole data to have min of 0 and max of 1 (ion based normalization)
def ion_minmax_normalize(peaks):
    max_ion_int = np.max(peaks, axis=0)
    min_ion_int = np.min(peaks, axis=0)
    peaks_ionorm = np.empty(peaks.shape)
    for i in range(peaks.shape[1]):
        if max_ion_int[i]!=min_ion_int[i]:
            peaks_ionorm[:,i] = (peaks[:,i]-min_ion_int[i])/(max_ion_int[i]-min_ion_int[i])
    return peaks_ionorm

In [ ]:
def GetBinnedData(BinSize, Data):
    #https://github.com/McDonnell-Lab/MALDI-UNISA-ML/blob/main/PrepareData_CRC.ipynb
    # Calculate the total number of bins needed to span the range of m/z values
    Overall_min_mzs = np.min(Data)
    Overall_max_mzs = np.max(Data)
    B = int((Overall_max_mzs - Overall_min_mzs) / BinSize) + 1
    
    # Define m/z bin boundaries that are linearly spaced between the minimum and maximum m/z
    BinEdges = np.linspace(Overall_min_mzs, Overall_max_mzs, B)
    
    # Assign each m/z point used to the nearest bin
    Mzs_binning_indices = np.digitize(Data, BinEdges)
    
    # Sum the intensities over each original m/z point assigned to each bin
    Binned_intensities = np.zeros((Data.shape[0], B))
    for i in range(Data.shape[0]):
        for j in range(Data.shape[1]):
            bin_idx = Mzs_binning_indices[i, j] - 1
            Binned_intensities[i, bin_idx] += Data[i, j]
    
    return Binned_intensities

In [ ]:
def logtransform(peaks):
    #https://pubs.acs.org/doi/full/10.1021/acs.analchem.2c00097?casa_token=ns41wFfcqrsAAAAA%3Ax_g4l8NCXWlBJ1EWD3jkxa33J0Z6fcFq85RfkEOP3eKwzgUhqfKXWZLWfn3eemRvhEkqBcDLhP1Qlfc    
    r = np.max(peaks)
    b = 0.01 / (1 - 0.01 * r)

    # log transform each intensity in whole data
    peaks_trans = np.empty(peaks.shape)
    for i in range(peaks.shape[1]):
        peaks_trans[:,i] = b * np.log((1 + peaks[:,i]) / b)
    return peaks_trans

## covert text DESI to numpy array

In [16]:
root_dir = r"C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon"

# Create a list to store the required information for each file
file_info_list = []

peaks_list = []

# Walk through root_dir
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if "colon" is in the directory name
    if 'colon' in dirpath:
        # If so, look for .txt files
        for filename in filenames:
            if 'colon' in filename:
                if filename.endswith('.txt'):
                    path = os.path.join(dirpath, filename)
                    [peaks, mz, dim_y, dim_x] = DESI_txt2numpy(path)
                    print(filename)
                    print(peaks.shape)
                    print(mz.shape)
                    print(dim_y, dim_x)
                    
                    peaks = tic_normalize(peaks)
                    peaks_norm = ion_minmax_normalize(peaks)
                    peaks_norm = peaks_norm.astype(float)
                    print(peaks_norm.shape)
                    
                    peaks_list.append(peaks_norm)

                    # Remove the .txt extension from the filename
                    #filename_without_ext = os.path.splitext(filename)[0]
                    
                    # Save peaks_norm as a np array in the same directory
                    #np.savetxt(os.path.join(dirpath, f"{filename_without_ext} array.txt"), peaks_image, delimiter=",")
                    
                    # Add the required information to the list as a dictionary
                    #file_info_list.append({'filename': filename_without_ext ,'peaks_shape': peaks.shape, 'mz_shape': mz.shape, 'dim_y': dim_y, 'dim_x': dim_x})
                    #print(file_info_list[-1])




file: C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 21 colon 0485699-1 Analyte 1\2021 03 21 colon 0485699-1 Analyte 1.txt
spectra: 44178


  0%|          | 0/44183 [00:00<?, ?it/s]

  0%|          | 0/44178 [00:00<?, ?it/s]

image dimension: 222 x 199
2021 03 21 colon 0485699-1 Analyte 1.txt
(44178, 2000)
(2000,)
222 199
(44178, 2000)
file: C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 22 colon 0741403-1 Analyte 1\2021 03 22 colon 0741403-1 Analyte 1.txt
spectra: 53823


  0%|          | 0/53828 [00:00<?, ?it/s]

...

  0%|          | 0/52328 [00:00<?, ?it/s]

image dimension: 211 x 248
2021 03 30 colon 0720931-3 Analyte 5.txt
(52328, 2000)
(2000,)
211 248
(52328, 2000)
file: C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 31 colon 1258561-2 Analyte 7\2021 03 31 colon 1258561-2 Analyte 7.txt
spectra: 45012


  0%|          | 0/45017 [00:00<?, ?it/s]

  0%|          | 0/45012 [00:00<?, ?it/s]

image dimension: 186 x 242
2021 03 31 colon 1258561-2 Analyte 7.txt
(45012, 2000)
(2000,)
186 242
(45012, 2000)


In [ ]:
for file_info in file_info_list:
    filename = file_info['filename'] + ' array.txt'
    dim_y = file_info['dim_y']
    dim_x = file_info['dim_x']
    output_name = filename.replace("array", "dc-DeepMSI")[:-4]
    print(f"python run.py --input_file \"C:\\Users\\jenni\\OneDrive - Queen's University\\DESI project\\DESI TXT colon\\DESI TXT colon\\{filename}\" --input_shape {dim_y} {dim_x} 2000 --mode spat-contig --output_file \"{output_name}\"")

In [ ]:
## read files, convert to matlab (or numpy), and save
data_path = r"C:\Users\jenni\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 21 colon 0485699-1 Analyte 1\2021 03 21 colon 0485699-1 Analyte 1.txt"
#slide_name = r"2021 03 21 colon 0485699-1 Analyte 1.txt"

#file = os.path.join(data_path,slide_name)
[peaks, mz, dim_y, dim_x] = DESI_txt2numpy(data_path)

print(peaks.shape)
print(mz.shape)
print(dim_y, dim_x)

np.savetxt("2021 03 22 colon 0741403-1 Analyte 1.csv", peaks, delimiter = ",")

In [17]:
def batch_peak_alignment(arrays):
    """
    OBJECTIVE:
    to align m/z values from multiple MSI slides
    DESCRIPTION:
    the function will receive a list of numpy arrays containing the peak intensities for alignment
    OUTPUT:
    aligned_peaks - a concatenated 2-dimensional array of aligned peaks
    """
    print('importing')
    # Import
    from scipy.signal import find_peaks

    mz, peaks = [], []
    for array in arrays:
        mz.append(array[-1])  # Assuming the last dimension is m/z
        peaks.append(array[:-1])  # All other dimensions are considered as peaks

    mz_all = np.sort(np.concatenate(mz)).astype('float64')

    ## cluster m/z values that are close to eachother
    print('accumulating m/z values...')
    mz_bandwidth = 0.01 #std of the estimated mz clusters
    abundance_threshold = 0.4 #eliminate the least abundant peaks between slides
    mz_resolution = 0.01 # m/z resolution for peak clustering
    nsig = 4 # number of std in kernel width

    # subband_start = 50
    # subband_stop = 1200 
    subband_start = np.round(mz_all.min()-nsig*mz_bandwidth, -1)-10  # 50 m/z range minimum
    subband_stop = np.round(mz_all.max()+nsig*mz_bandwidth, -1)+10  # 1200 m/z range maximum
    
    # generate sample Gaussian kernel
    x = np.arange(-nsig*mz_bandwidth,nsig*mz_bandwidth+1e-5,mz_resolution)
    x = np.round(x, decimals=4)
    gkerbel = np.exp(-0.5*(x/mz_bandwidth)**2)

    mz_grid = np.arange(subband_start-1,subband_stop+1,mz_resolution)
    mz_grid = np.round(mz_grid, decimals=4)

    # find the kernel density distribution of all m/z values
    kde_grid = np.zeros(mz_grid.shape)
    width = int(len(gkerbel)/2)
    for mzi in mz_all:
        ind = np.argmin(np.abs(mz_grid-mzi))
        kde_grid[ind-width:ind+width+1]+=gkerbel # 3x faster than convolution 

    # find the peak values as the selected m/z
    locs, _ = find_peaks(kde_grid, height=0.5)

    selected_mz = mz_grid[locs]
    num_mz = len(selected_mz)

    ## estimate the contribution of each slide in selected m/z
    print('estimating the contribution of each csv...')
    n_mz = len(selected_mz)
    n_csv = len(mz)
    new_mz = []

    for i in range(n_csv):
        x=[]
        for j in range(n_mz):
            x.append([])
        new_mz.append(x)    

    for i,old_mz in enumerate(mz):
        for j,x in enumerate(old_mz):
            diff = np.abs(selected_mz-x)
            ind = diff.argmin()
            new_mz[i][ind].append(x)

    mz_df = pd.DataFrame(new_mz, columns=selected_mz)
    
    # eliminate the least repeated mz 
    num_sample_per_slide = np.array([x.shape[0] for x in peaks]).T

    num_nonnan_mz = []
    for j in range(n_mz):
        x=0
        for i in range(n_csv):
            x += ( len(new_mz[i][j])>0 )*num_sample_per_slide[i]
        num_nonnan_mz.append(x)

    nonnan_threshold = int(abundance_threshold*np.sum(num_sample_per_slide))

    kept_mz_ind = np.array(num_nonnan_mz) > nonnan_threshold
    n_mz = kept_mz_ind.sum()
    print('number of selected m/z:', n_mz)

    new_mz_df = mz_df[mz_df.columns[kept_mz_ind]]
    
    # eliminate multiple m/z values
    final_mz_df = pd.DataFrame(np.nan*np.ones([n_csv,n_mz]), columns=new_mz_df.columns)
    for i in range(n_csv):
        mz_old = mz[i]
        pmean = peaks[i].mean(axis=0)/peaks[i].mean(axis=0).max()
        for j in range(n_mz):
            mz_cell = new_mz_df[new_mz_df.columns[j]][i]
            if len(mz_cell)==0:
                pass
            elif len(mz_cell)==1:
                final_mz_df[final_mz_df.columns[j]][i] = mz_cell[0]
            else:
                ind = [np.abs(x-mz_old).argmin() for x in mz_cell]
                i_abundant = pmean[ind].argmax()
                final_mz_df[final_mz_df.columns[j]][i] = mz_cell[i_abundant]
                            

    # correct the selected mz
    mz_list = final_mz_df.to_numpy()
    mz_selected_old = final_mz_df.columns
    corrected_mz = []
    for i in range( n_mz ):
        col = mz_list[:,i]
        col = col[~np.isnan(col)]
        min_i = np.abs(col-mz_selected_old[i]).argmin()
        corrected_mz.append( col[min_i] )

    corrected_mz = np.array(corrected_mz)
    final_mz_df.columns = corrected_mz

    print('align peaks')
    # align peaks
    aligned_peaks = []
    for i in range(n_csv):
        mz_old = mz[i]
        newp = np.nan * np.ones([len(peaks[i]), n_mz])
        for j, col in enumerate(corrected_mz):
            mz_val = final_mz_df[col][i]
            if ~np.isnan(mz_val):
                p_ind = np.where(mz_old == mz_val)[0]

                print(peaks[i][:, p_ind].flatten().shape)
                print(newp[:, j].shape)

                newp[:, j] = peaks[i][:, p_ind].flatten()[:len(newp[:, j])]
        aligned_peaks.append(newp)

    aligned_peaks = np.concatenate(aligned_peaks, axis=0)

    print(aligned_peaks.shape)

    np.save('aligned_peaks.npy', aligned_peaks)


In [11]:
## read files, convert to matlab (or numpy), and save
data_path = r"C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 21 colon 0485699-1 Analyte 1\2021 03 21 colon 0485699-1 Analyte 1.txt"
data_path2 = r"C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 22 colon 0741403-1 Analyte 1\2021 03 22 colon 0741403-1 Analyte 1.txt"

#file = os.path.join(data_path,slide_name)
[peaks, mz, dim_y, dim_x] = DESI_txt2numpy(data_path)

print(peaks.shape)
print(mz.shape)
print(dim_y, dim_x)

#file = os.path.join(data_path,slide_name)
[peaks2, mz2, dim_y2, dim_x2] = DESI_txt2numpy(data_path2)

print(peaks2.shape)
print(mz2.shape)
print(dim_y2, dim_x2)

peaks_list = [peaks, peaks2]

file: C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 21 colon 0485699-1 Analyte 1\2021 03 21 colon 0485699-1 Analyte 1.txt
spectra: 44178


  0%|          | 0/44183 [00:00<?, ?it/s]

  0%|          | 0/44178 [00:00<?, ?it/s]

image dimension: 222 x 199
(44178, 2000)
(2000,)
222 199
file: C:\Users\tsang\OneDrive - Queen's University\DESI project\DESI TXT colon\DESI TXT colon\2021 03 22 colon 0741403-1 Analyte 1\2021 03 22 colon 0741403-1 Analyte 1.txt
spectra: 53823


  0%|          | 0/53828 [00:00<?, ?it/s]

  0%|          | 0/53823 [00:00<?, ?it/s]

image dimension: 231 x 233
(53823, 2000)
(2000,)
231 233


In [18]:
batch_peak_alignment(peaks_list)

importing
accumulating m/z values...
estimating the contribution of each csv...
number of selected m/z: 6
align peaks
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(44177,)
(53822,)
(53822,)
(53822,)
(53822,)
(53822,)
(53822,)
(53822,)
(53822,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(43474,)
(51979,)
(51979,)
(51979,)
(51979,)
(51979,)
(51979,)
(51979,)
(51979,)
(93791081,)
(46919,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(47501,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(50499,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(53914,)
(52327,)
(52327,)
(52327,)
(52327,)
(52327,)
(52327,)
(52327,)
(52327,)
(90022000,)
(45011,)
(489623, 6)


In [ ]:
import re

def correct_scientific_notation(s):
    """Corrects the scientific notation in a string."""
    if isinstance(s, str):
        return re.sub(r'(\d+\.\d+e\+\d+)\.\d+', r'\1', s)
    return s

In [ ]:
def batch_peak_alignments(arrays, csv_save_name):
		""""
		OBJECTIVE:
		to align m/z values from muliple MSI slides
		DESCRIPTION:
		the finction will receive the location of the folder that contains all MSI csv data for alignment and svae the aligned csv in the same folder
		INPUTS:
		data_dir - the name of the folder that conains the individual MSI csv files 
		csv_save_name - the name of the final aligned file
		AUTHOR:
		@Moon
		"""
		# Import
		from scipy.signal import find_peaks
		
		mz, peaks = [], []
		for array in arrays:
			mz.append(array[-1])  # Assuming the last dimension is m/z
			peaks.append(array[:-1])  # All other dimensions are considered as peaks
		print('importing')

		mz_all = np.sort(np.concatenate(mz)).astype('float64')
		
		## cluster m/z values that are close to eachother
		print('accumulating m/z values...')
		mz_bandwidth = 0.01 #std of the estimated mz clusters
		abundance_threshold = 0.4 #eliminate the least abundant peaks between slides
		mz_resolution = 0.01 # m/z resolution for peak clustering
		nsig = 4 # number of std in kernel width

		# subband_start = 50
		# subband_stop = 1200 
		subband_start = np.round(mz_all.min()-nsig*mz_bandwidth, -1)-10  # 50 m/z range minimum
		subband_stop = np.round(mz_all.max()+nsig*mz_bandwidth, -1)+10  # 1200 m/z range maximum
		
		# generate sample Gaussian kernel
		x = np.arange(-nsig*mz_bandwidth,nsig*mz_bandwidth+1e-5,mz_resolution)
		x = np.round(x, decimals=4)
		gkerbel = np.exp(-0.5*(x/mz_bandwidth)**2)

		mz_grid = np.arange(subband_start-1,subband_stop+1,mz_resolution)
		mz_grid = np.round(mz_grid, decimals=4)

		# find the kernel density distribution of all m/z values
		kde_grid = np.zeros(mz_grid.shape)
		width = int(len(gkerbel)/2)
		for mzi in mz_all:
			ind = np.argmin(np.abs(mz_grid-mzi))
			kde_grid[ind-width:ind+width+1]+=gkerbel # 3x faster than convolution 

		# find the peak values as the selected m/z
		locs, _ = find_peaks(kde_grid, height=0.5)

		selected_mz = mz_grid[locs]
		num_mz = len(selected_mz)

		## estimate the contribution of each slide in selected m/z
		print('estimating the contribution of each csv...')
		n_mz = len(selected_mz)
		n_csv = len(mz)
		new_mz = []

		for i in range(n_csv):
			x=[]
			for j in range(n_mz):
				x.append([])
			new_mz.append(x)    

		for i,old_mz in enumerate(mz):
			for j,x in enumerate(old_mz):
				diff = np.abs(selected_mz-x)
				ind = diff.argmin()
				new_mz[i][ind].append(x)

		mz_df = pd.DataFrame(new_mz, columns=selected_mz)
		
		# eliminate the least repeated mz 
		num_sample_per_slide = np.array([x.shape[0] for x in peaks]).T

		num_nonnan_mz = []
		for j in range(n_mz):
			x=0
			for i in range(n_csv):
				x += ( len(new_mz[i][j])>0 )*num_sample_per_slide[i]
			num_nonnan_mz.append(x)

		nonnan_threshold = int(abundance_threshold*np.sum(num_sample_per_slide))

		kept_mz_ind = np.array(num_nonnan_mz) > nonnan_threshold
		n_mz = kept_mz_ind.sum()
		print('number of selected m/z:', n_mz)

		new_mz_df = mz_df[mz_df.columns[kept_mz_ind]]
		
		# eliminate multiple m/z values
		final_mz_df = pd.DataFrame(np.nan*np.ones([n_csv,n_mz]), columns=new_mz_df.columns)
		for i in range(n_csv):
			mz_old = mz[i]
			pmean = peaks[i].mean(axis=0)/peaks[i].mean(axis=0).max()
			for j in range(n_mz):
				mz_cell = new_mz_df[new_mz_df.columns[j]][i]
				if len(mz_cell)==0:
					pass
				elif len(mz_cell)==1:
					final_mz_df[final_mz_df.columns[j]][i] = mz_cell[0]
				else:
					ind = [np.abs(x-mz_old).argmin() for x in mz_cell]
					i_abundant = pmean[ind].argmax()
					final_mz_df[final_mz_df.columns[j]][i] = mz_cell[i_abundant]
								

		# correct the selected mz
		mz_list = final_mz_df.to_numpy()
		mz_selected_old = final_mz_df.columns
		corrected_mz = []
		for i in range( n_mz ):
			col = mz_list[:,i]
			col = col[~np.isnan(col)]
			min_i = np.abs(col-mz_selected_old[i]).argmin()
			corrected_mz.append( col[min_i] )

		corrected_mz = np.array(corrected_mz)
		final_mz_df.columns = corrected_mz

		# align peaks
		aligned_peaks = []
		for i in range(n_csv):
			mz_old = mz[i]
			newp = np.nan*np.ones([len(peaks[i]),n_mz])
			for j,col in enumerate(corrected_mz):
				mz_val = final_mz_df[col][i]
				if ~np.isnan(mz_val):
					p_ind = np.where(mz_old == mz_val)[0]
					if p_ind.size > 0:
						newp[:,j] = np.mean(peaks[i][:,p_ind], axis=1)
			aligned_peaks.append(newp)

		aligned_peaks = np.concatenate(aligned_peaks, axis=0)
		print(aligned_peaks.shape)

		# save aligned peaks
		csv_data = aligned_peaks
		csv_column = list(corrected_mz)
		df = pd.DataFrame(csv_data, columns=csv_column)
		print('saving the aligned peaks csv...')
		df.to_csv(csv_save_name, index=False)
		print('alignment done!')

		### report information
		retstr = 'Dataset successfully aligned! \n'
		retstr += f'Aligned dataset:\t {csv_save_name} \n'
		return retstr

In [ ]:
csv_list = [r"C:\Users\jenni\OneDrive - Queen's University\DESI project\DESI TXT colon\stuff\Arrays csv\2021 03 21 colon 0485699-1 Analyte 1 array.csv", r"C:\Users\jenni\OneDrive - Queen's University\DESI project\DESI TXT colon\stuff\Arrays csv\2021 03 22 colon 0741403-1 Analyte 1 array.csv", r"C:\Users\jenni\OneDrive - Queen's University\DESI project\DESI TXT colon\stuff\Arrays csv\2021 03 22 colon 0962695-1 Analyte 1 array.csv"]

## preprocessing

In [ ]:
peaks = tic_normalize(peaks)
peaks_norm = ion_minmax_normalize(peaks)
print(peaks_norm.shape)

#np.savetxt("2021 03 22 colon 0741403-1 Analyte 1.txt", peaks_norm)


In [ ]:
peaks_log = logtransform(peaks_norm)
print(peaks_log[1:4])
print(peaks_log.shape)

In [ ]:
dz = 0.001
print(max_ion_int, min_ion_int)

peaks_binned = GetBinnedData(dz, peaks_norm)
#print(peaks_binned[1:4])
print(peaks_binned.shape)

In [ ]:
np.savetxt("2021 03 22 colon 0741403-1 log10.txt", peaks_log)

## visualize single spectrum and single ion image

In [ ]:

# reshape peaks to 3D (dim_y x dim_x x N_ions)
peaks_image = peaks.reshape((dim_y,dim_x,-1),order='C')
print(peaks_image.shape)

#visualize a spectrum from a specific (x,y) location
#ind_y = 100
#ind_x = 170
ind_y = dim_y - 1
ind_x = dim_x - 1
plt.figure(figsize=(10,3))
plt.stem(mz, peaks_image[ind_y,ind_x,:], linefmt='k', markerfmt='k', basefmt='k')
plt.xlabel('m/z')
plt.ylabel('intensity')
plt.title('mass spectrum at '+str([ind_y, ind_x]))
plt.show()

# visualize the peak intensity of a specific ion for all spectra
ind_mz = 234
plt.figure(figsize=(8,8))
plt.imshow( peaks_image[:,:,ind_mz], cmap='plasma')
plt.axis('off')
plt.title('m/z = '+str(mz[ind_mz]))
plt.show()


## univariante combined visualization

In [ ]:
# mz[np.argmin(np.abs(mz-277))]

mz_r = 726.539
mz_g = 766.5359
mz_b = 277.0737

# ch_r = np.argwhere(mz==mz_r)
# ch_g = np.argwhere(mz==mz_g)
# ch_b = np.argwhere(mz==mz_b)

ch_r = np.argmin(np.abs(mz-mz_r))
ch_g = np.argmin(np.abs(mz-mz_g))
ch_b = np.argmin(np.abs(mz-mz_b))

image_r = peaks_norm[:,ch_r].reshape((dim_y,dim_x,-1),order='C')
image_g = peaks_norm[:,ch_g].reshape((dim_y,dim_x,-1),order='C')
image_b = peaks_norm[:,ch_b].reshape((dim_y,dim_x,-1),order='C')
image = np.concatenate([image_r,image_g,image_b], axis=2)

int_scale = 2
plt.figure(figsize=(8,8))
plt.imshow(  np.clip(int_scale*image,0,1) , cmap='gray')
plt.show()

## multivaiant visualization

In [ ]:
dim_reducion = PCA(n_components=3)

peaks_pca = dim_reducion.fit_transform(peaks_norm)
peaks_pca = ion_minmax_normalize(peaks_pca)
pca_image = peaks_pca.reshape((dim_y,dim_x,-1),order='C')

plt.figure(figsize=(8,8))
plt.imshow(pca_image)
plt.show()

In [ ]:
#np.savetxt("2021 03 22 colon 0741403-1 PCA.txt", peaks_pca)
print(pca_image.shape)

In [ ]:
dim_reduction = PCA(n_components=3)

mask = sitk.ReadImage(r"C:\Users\jenni\OneDrive\Documents\DESI-Sampledata\2021 03 21 colon 0485699_mask.png")
mask_array = sitk.GetArrayFromImage(mask)

mask_reshaped = mask_array[:, :, 0].reshape((222, 199, 1))
print(mask_reshaped[:, :, 0].shape) #testing

peaks_image = peaks_norm.reshape((dim_y,dim_x,-1),order='C') #reshape peaks_norm to (222, 199, 2000)
print(peaks_image.shape) #testing

selected_spectra = peaks_image * mask_reshaped #try index instead to select areas

num_rows, num_cols, num_channels = peaks_image.shape

dim_reduction.fit(peaks_norm)

pca_results = np.zeros((num_rows, num_cols, dim_reduction.n_components))  # Initialize the result array

for i in range(num_rows):
    for j in range(num_cols):
        if mask_reshaped[i, j] == 255:
            # Extract the region of interest (ROI) from the MSI data
            roi = peaks_image[i, j, :].reshape(1, -1)  # Reshape to (1, num_channels)

            # Perform PCA on the ROI using the pre-fitted PCA model
            pca_result = dim_reduction.transform(roi)
            pca_result = ion_minmax_normalize(pca_result)

            # Store the PCA results in the corresponding location in the array
            pca_results[i, j, :] = pca_result

plt.figure(figsize=(8,8))
plt.imshow(pca_results)
plt.show()

In [ ]:
dim_reduction = PCA(n_components=3)

mask = sitk.ReadImage(r"C:\Users\jenni\OneDrive\Documents\DESI-Sampledata\2021 03 21 colon 0485699_mask.png")
mask_array = sitk.GetArrayFromImage(mask)

peaks_image = peaks_norm.reshape((dim_y,dim_x,-1),order='C') #reshape peaks_norm to (222, 199, 2000)
print(peaks_image.shape) #testing

#Get peaks_image shape information
num_rows, num_cols, num_channels = peaks_image.shape


# Step 1: Identify the ROI using the binary mask and create list of ij locations
roi_indices = np.where(mask_array == 255)
roi_pixels = list(zip(roi_indices[0], roi_indices[1]))

# Step 2: Extract spectra from ROI pixels
roi_spectra = []
for i, j in roi_pixels:
    # Extract the spectra for the current ROI pixel (i, j)
    pixel_spectra = peaks_image[i, j]
    roi_spectra.append(pixel_spectra)

# Step 3: convert to np array
roi_spectra_array = np.array(roi_spectra)

# Step 4: Perform PCA

# fit PCA
dim_reduction.fit(roi_spectra_array)

# Perform PCA and normalize
roi_pca = dim_reduction.transform(roi_spectra_array)
roi_pca = ion_minmax_normalize(roi_pca)

# Create empty array and map PCA results from ROI back to the original dimensions (222, 199, 3)
pca_results = np.zeros((num_rows, num_cols, dim_reduction.n_components))
for idx, (i, j) in enumerate(roi_pixels):
    pca_results[i, j, :] = roi_pca[idx]

# # Plot results
# plt.figure(figsize=(8, 8))
# plt.imshow(pca_results)
# plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(pca_results, cmap='viridis')
axes[0].set_title('PCA w/ Mask')

axes[1].imshow(pca_image, cmap='viridis')
axes[1].set_title('PCA Original')

plt.show()


In [ ]:
dim_reducion = PCA(n_components=3)

mask = sitk.ReadImage(r"C:\Users\jenni\OneDrive\Documents\DESI-Sampledata\2021 03 21 colon 0485699_mask.png")
mask_array = sitk.GetArrayFromImage(mask)


mask_reshaped = mask_array[:, :, 0].reshape((222, 199, 1))
print(mask_array[:, :, 0].shape)

peaks_image = peaks_norm.reshape((dim_y,dim_x,-1),order='C') #reshape peaks_norm to (222, 199, 2000)
print(peaks_image.shape) #testing

selected_spectra = peaks_image * mask_reshaped #try index instead to select areas

print(selected_spectra.shape)
#try making a for loop to select pixels that are part of the mask and not, append the ij location of them. New array with PCA results
selected_spectra_image = selected_spectra.reshape(dim_y*dim_x, -1) #Reshape image back to 2D array dim_y*dim_x, -1
print(selected_spectra_image.shape) #testing

peaks_pca = dim_reducion.fit_transform(selected_spectra_image)
peaks_pca = ion_minmax_normalize(peaks_pca)

print(peaks_pca.shape)
pca_image = peaks_pca.reshape((222,199,-1),order='C')
print(pca_image.shape)

plt.figure(figsize=(8,8))
plt.imshow(pca_image)
plt.show()